## 🔗 Open This Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DavidLangworthy/ds4s/blob/master/days/day02/notebook/day02_starter.ipynb)

# ⚡ Day 2 – Tracking the Renewable Energy Transition
We continue with short learn–do cycles. Each section nudges the data a little further so you can constantly confirm understanding before layering more complexity.

## 🗂️ Data Card: Global Energy Mix
- **Source:** Our World in Data, based on BP Statistical Review and IEA conversions.
- **Temporal coverage:** 1965–2021, annual.
- **Units:** Share of primary energy from renewables (%). Separate files track hydro, wind, solar.
- **Last updated:** July 2022 (snapshotted for this course).
- **Method notes:** Energy expressed as share of total primary energy demand using BP's substitution method.
- **Caveats:** Country definitions follow BP; shares can exceed 100% in edge cases if totals are revised.
- **Integrity prompt:** How might stacked area choices mislead (e.g., order of layers, color saturation)? Commit to one tactic that keeps interpretation honest.

## Story Scaffold Reminder
- **Claim:** What shift in the energy mix are you highlighting?
- **Evidence:** Which metrics show that shift?
- **Visual:** How do color, ordering, and axes make the comparison fair?
- **Takeaway:** Draft the sentence you want a policymaker to repeat.

## Step 0 · Imports and styling

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from days.utils import (
    add_story_footer,
    baseline_style,
    check_story_metadata,
    load_data,
    quick_diagnostics,
    save_last_fig,
)

baseline_style()
sns.set_palette(["#2F4B7C", "#59A14F", "#F28E2B", "#E15759"])

## Step 1 · Load the renewable datasets
Focus on the world totals so you can compare the mix over time without worrying about country-level differences yet.

In [ ]:
total = load_data("data/01 renewable-share-energy.csv")
hydro = load_data("data/06 hydro-share-energy.csv")
wind = load_data("data/10 wind-share-energy.csv")
solar = load_data("data/14 solar-share-energy.csv")

world_filters = total["Entity"] == "World"
renewables_world = (
    total.loc[world_filters, ["Year", "Renewables (% equivalent primary energy)"]]
    .rename(columns={"Renewables (% equivalent primary energy)": "RenewableShare"})
)
renewables_world = renewables_world.merge(
    hydro.loc[world_filters, ["Year", "Hydro (% equivalent primary energy)"]],
    on="Year",
).merge(
    wind.loc[world_filters, ["Year", "Wind (% equivalent primary energy)"]],
    on="Year",
).merge(
    solar.loc[world_filters, ["Year", "Solar (% equivalent primary energy)"]],
    on="Year",
)
renewables_world = renewables_world.rename(
    columns={
        "Hydro (% equivalent primary energy)": "Hydro",
        "Wind (% equivalent primary energy)": "Wind",
        "Solar (% equivalent primary energy)": "Solar",
    }
).sort_values("Year").reset_index(drop=True)

In [ ]:
quick_diagnostics(
    renewables_world,
    expected_columns=["Year", "RenewableShare", "Hydro", "Wind", "Solar"],
    rows_between=(50, 60),
)
print("Expected: 57 annual rows for 1965–2021 with percentage values between 0 and ~30.")

## Step 2 · Prepare long-form data for stacking
Transform the wide table into a tidy layout so you can reuse it for multiple charts and calculations.

In [ ]:
composition_long = renewables_world.melt(
    id_vars=["Year", "RenewableShare"],
    value_vars=["Hydro", "Wind", "Solar"],
    var_name="Source",
    value_name="Share",
)")

In [ ]:
quick_diagnostics(
    composition_long.head(15),
    expected_columns=["Year", "RenewableShare", "Source", "Share"],
    rows_between=(150, 200),
    head_rows=5,
)
print("Check: three rows per year — one for each source.")

## Step 3 · Interim progress check
Plot the total renewable share to confirm the overall trajectory before layering the individual technologies.

![Interim preview – line chart climbing from 5% to ~15%.](../../plots/day02_solution_plot.png)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(renewables_world["Year"], renewables_world["RenewableShare"], color="#2F4B7C", linewidth=2)
ax.set_xlabel("Year")
ax.set_ylabel("Share of global primary energy from renewables (%)")
ax.set_title("Draft: Renewable share is steadily rising")
ax.set_ylim(0, max(renewables_world["RenewableShare"]) + 5)
plt.show()

## Step 4 · Story metadata

In [ ]:
TITLE = "Renewables tripled their share, but the race is far from done"
SUBTITLE = "World primary energy mix, 1965–2021"
ANNOTATION = "Wind and solar now supply nearly as much energy as hydropower combined."
SOURCE = "Source: Our World in Data (BP Statistical Review 2022 edition)"
UNITS = "Units: Share of total primary energy (%)"

check_story_metadata(
    TITLE=TITLE,
    SUBTITLE=SUBTITLE,
    ANNOTATION=ANNOTATION,
    SOURCE=SOURCE,
    UNITS=UNITS,
)

## Step 5 · Final stacked area story
Layer the renewable sources, annotate the combined impact, and retain a single y-axis to keep interpretation honest.

In [ ]:
fig, ax = plt.subplots(figsize=(11, 6))
sources = ["Hydro", "Wind", "Solar"]
colors = ["#2F4B7C", "#59A14F", "#F28E2B"]
ax.stackplot(
    renewables_world["Year"],
    [renewables_world[src] for src in sources],
    labels=sources,
    colors=colors,
    alpha=0.85,
)
ax.plot(renewables_world["Year"], renewables_world["RenewableShare"], color="#1D3557", linewidth=2.5, label="Total renewable share")
ax.set_title(f"{TITLE}\n{SUBTITLE}")
ax.set_xlabel("Year")
ax.set_ylabel("Share of global primary energy (%)")
ax.set_ylim(0, max(renewables_world["RenewableShare"]) + 5)
ax.legend(loc="upper left", frameon=False)
latest = renewables_world.iloc[-1]
ax.annotate(
    ANNOTATION,
    xy=(latest["Year"], latest["RenewableShare"]),
    xytext=(latest["Year"] - 15, latest["RenewableShare"] + 5),
    arrowprops=dict(arrowstyle="->", color="#1D3557"),
    fontsize=11,
    color="#1D3557",
)
add_story_footer(ax, source=SOURCE, units=UNITS)
plt.show()

## Step 6 · Interpret using the scaffold
- **Claim:** Renewable energy's share of global primary demand has climbed from ~5% to ~15%, powered by wind and solar gains.
- **Evidence:** Hydro dominated until the 2000s; wind and solar contributions accelerate after 2010, pushing the total curve upward.
- **Visual:** A stackplot with a contrasting outline reinforces that the total share is what matters for climate goals.
- **Takeaway:** “Wind and solar growth finally matches hydropower, but renewables still supply barely one-sixth of the world’s energy — we need faster scaling.”

### What the chart does not show
- Fossil fuel absolute consumption is still rising; add a companion chart if discussing emissions.
- Shares hide efficiency and demand trends — consider energy intensity or total TWh for deeper analysis.
- Country-level disparities matter for policy; suggest an extension where students explore leaders and laggards.

## Step 7 · Save the output

In [ ]:
save_last_fig("day02_solution_plot.png")